In [1]:
import numpy as np 
import pandas as pd 

# MEAN

In [3]:
array= np.array([100,150,200,250,360,490,500,600,671])
array.mean()

369.0

# MEDIAN

In [4]:
array= np.array([25,18,27,10,8,30,42,20,53])
np.median(array)

25.0

# MODE

In [5]:
array= np.array([1,1,2,3,4,5])
data= pd.Series(array) 
data.mode()

0    1
dtype: int32

# STANDARD DEVIATION

In [6]:
array= np.array([1,1,2,3,4,5])
data= pd.Series(array)
data.std()

1.632993161855452

# QUARTILES

In [7]:
array= np.array([25,18,30, 8, 15, 5, 10, 35, 40, 45])
data= pd.Series(array)
data.describe()

count    10.000000
mean     23.100000
std      14.051097
min       5.000000
25%      11.250000
50%      21.500000
75%      33.750000
max      45.000000
dtype: float64

# Quartile deviation

In [8]:
qd= (data.describe()[6]-data.describe()[4])/2
print("(q3-q1)/2= ", qd)

(q3-q1)/2=  11.25


# Coefficient of quartile deviation

In [9]:
coff= (data.describe()[6]-data.describe()[4])/(data.describe()[6]+data.describe()[4])
print("(q3-q1)/(q3+q1)= ", coff)

(q3-q1)/(q3+q1)=  0.5


# MEAN DEVIATION

In [10]:
array= np.array([100,150,200,250,360,490,500,600,671])
data= pd.Series(array)
mean= data.mean()
md= data.mad()
md

174.44444444444446

# Co efficient of mean deviation

In [11]:
comd= md/mean
comd

0.47274917193616384

# Newton Gregory forward & Backward Interploatoin Formula

In [29]:
def u_cal(u, n): 
	temp = u; 
	for i in range(1, n): 
		temp = temp * (u - i)
	return temp; 

def fact(n): 
	f = 1 
	for i in range(2, n + 1): 
		f *= i
	return f 
n = int(input("Enter n values"))
x= []
for i in range(0,n):
    ele= float(input("Enter x values"))
    x.append(ele)

print("X Values:")
print(x)
print()

y = [[0 for i in range(n)] 
		for j in range(n)]
for i in range(0,n):
    y[i][0] = float(input("Enter y")) 
print("Y Values:")
print(y)
print()

for i in range(1, n): 
	for j in range(n - i): 
		y[j][i] = y[j + 1][i - 1] - y[j][i - 1]

for i in range(n): 
	print(x[i], end = "\t") 
	for j in range(n - i): 
		print(y[i][j], end = "\t") 
	print("") 

value = float(input("enter the value you need to find at:"))
sum = y[0][0] 
u = (value - x[0]) / (x[1] - x[0]) 
for i in range(1,n): 
	sum = sum + (u_cal(u, i) * y[0][i]) / fact(i)

print("\nValue at", value, 
	"is", round(sum, 6))


X Values:
[5.0, 7.0, 11.0, 13.0, 17.0]

Y Values:
[[150.0, 0, 0, 0, 0], [392.0, 0, 0, 0, 0], [1452.0, 0, 0, 0, 0], [2366.0, 0, 0, 0, 0], [5202.0, 0, 0, 0, 0]]

5.0	150.0	242.0	818.0	-964.0	3032.0	
7.0	392.0	1060.0	-146.0	2068.0	
11.0	1452.0	914.0	1922.0	
13.0	2366.0	2836.0	
17.0	5202.0	

Value at 9.0 is 1452.0


# Lagranges' Interploation Formula for unequal intervals

In [104]:
n = int(input('Enter number of data points: '))
x = np.zeros((n))
y = np.zeros((n))
print('Enter data for x and y: ')
for i in range(n):
    x[i] = float(input( 'x['+str(i)+']='))
print(x)
for i in range(n):
    y[i] = float(input( 'y['+str(i)+']='))
print(y)

xp = float(input('Enter interpolation point: '))
yp = 0

for i in range(n): 
    p = 1 
    for j in range(n):
        if i != j:
            p = p * (xp - x[j])/(x[i] - x[j])    
    yp = yp + p * y[i]    

print('Interpolated value at %.3f is %.3f.' % (xp, yp))

Enter data for x and y: 
[0. 2. 3. 6.]
[ -4.   2.  14. 158.]
Interpolated value at 4.000 is 40.000.


# Polynominal equations based on interpolations

In [37]:
from sympy import simplify, symbols, expand, factor, sin, cos #..., lambdify, solve_poly_system

In [90]:
%%writefile LagrangePolynomial.py
"""
From: https://gist.github.com/folkertdev/084c53887c49a6248839
A sympy-based Lagrange polynomial constructor. 

Implementation of Lagrangian interpolating polynomial.
See:

   def lagrangePolynomial(xs, ys):

Given two 1-D arrays `xs` and `ys,` returns the Lagrange interpolating
polynomial through the points ``(xs, ys)``


Given a set 1-D arrays of inputs and outputs, the lagrangePolynomial function 
will construct an expression that for every input gives the corresponding output. 
For intermediate values, the polynomial interpolates (giving varying results 
based  on the shape of your input). 

The Lagrangian polynomials can be obtained explicitly with (see below):
   
   def polyL(xs,j):
   
as sympy polynomial, and 

    def L(xs,j):

as Python functions.


This is useful when the result needs to be used outside of Python, because the 
expression can easily be copied. To convert the expression to a python function 
object, use sympy.lambdify.
"""
from sympy import symbols, expand, lambdify, solve_poly_system
#Python library for arithmetic with arbitrary precision
from mpmath import tan, e

import math

from operator import mul
from functools import reduce, lru_cache
from itertools import chain

# sympy symbols
x = symbols('x')

# convenience functions
product = lambda *args: reduce(mul, *(list(args) + [1]))

# test data
labels = [(-3/2), (-3/4), 0, 3/4, 3/2]
points = [math.tan(v) for v in labels]

# this product may be reusable (when creating many functions on the same domain)
# therefore, cache the result
@lru_cache(16)
def l(labels, j):
    def gen(labels, j):
        k = len(labels)
        current = labels[j]
        for m in labels:
            if m == current:
                continue
            yield (x - m) / (current - m)
    return expand(product(gen(labels, j)))

def polyL(xs,j):
    '''
    Lagrange polynomials as sympy polynomial
    xs: the n+1 nodes of the intepolation polynomial in the Lagrange Form
    j: Is the j-th Lagrange polinomial for the specific xs.
    '''
    xs=tuple(xs)
    return l(xs,j)

def L(xs,j):
    '''
    Lagrange polynomials as python function
    xs: the n+1 nodes of the intepolation polynomial in the Lagrange Form
    j: Is the j-th Lagrange polinomial for the specific xs.
    '''
    return lambdify(x, polyL(xs,j) )

def lagrangePolynomial(xs, ys):
    '''
    Given two 1-D arrays `x` and `w,` returns the Lagrange interpolating
    polynomial through the points ``(x, w)``.

    '''
    # based on https://en.wikipedia.org/wiki/Lagrange_polynomial#Example_1
    k = len(xs)
    total = 0

    # use tuple, needs to be hashable to cache
    xs = tuple(xs)

    for j, current in enumerate(ys):
        t = current * l(xs, j)
        total += t

    return total




def x_intersections(function, *args):
    "Finds all x for which function(x) = 0"
    # solve_poly_system seems more efficient than solve for larger expressions
    return [var for var in chain.from_iterable(solve_poly_system([function], *args)) if (var.is_real)]

def x_scale(function, factor):
    "Scale function on the x-axis"
    return functions.subs(x, x / factor)

if __name__ == '__main__':
    func = lagrangePolynomial(labels, points)

    pyfunc = lambdify(x, func)

    for a, b in zip(labels, points):
        assert(pyfunc(a) - b < 1e-6)

Overwriting LagrangePolynomial.py


In [91]:
%pylab inline
import pandas as pd
import numpy as np
import LagrangePolynomial as LP
from scipy import interpolate

Populating the interactive namespace from numpy and matplotlib


In [101]:
n=int(input("Enter the n values"))
x = np.zeros((n))
y = np.zeros((n))
for i in range(0,n):
    x[i]=float(input("Enter the x values"))
for i in range(0,n):
    y[i]=float(input("Enter the y values"))
print("X values:")
print(x)
print()

print("Y values:")
print(y)
print()

X values:
[1. 2. 3. 4. 5.]

Y values:
[ 1. -1.  1. -1.  1.]



In [102]:
df=pd.DataFrame({ 'X':x,'Y':y})
df

,X,Y
0,1.0,1.0
1,2.0,-1.0
2,3.0,1.0
3,4.0,-1.0
4,5.0,1.0


In [84]:
interpolate.lagrange?

Signature: interpolate.lagrange(x, w)
Docstring:
Return a Lagrange interpolating polynomial.

Given two 1-D arrays `x` and `w,` returns the Lagrange interpolating
polynomial through the points ``(x, w)``.

be able to use more than about 20 points even if they are chosen optimally.

Parameters
----------
x : array_like
    `x` represents the x-coordinates of a set of datapoints.
w : array_like
    `w` represents the y-coordinates of a set of datapoints, i.e., f(`x`).

Returns
-------
lagrange : `numpy.poly1d` instance
    The Lagrange interpolating polynomial.

Examples
--------
Interpolate :math:`f(x) = x^3` by 3 points.

>>> from scipy.interpolate import lagrange
>>> x = np.array([0, 1, 2])
>>> y = x**3
>>> poly = lagrange(x, y)

Since there are only 3 points, Lagrange polynomial has degree 2. Explicitly,
it is given by

.. math::

    \begin{aligned}
        L(x) &= 1\times \frac{x (x - 2)}{-1} + 8\times \frac{x (x-1)}{2} \\
             &= x (-2 + 3x)
    \end{aligned}

>>> from num

In [103]:
P=interpolate.lagrange(df.X,df.Y)
print(P)

        4     3         2
0.6667 x - 8 x + 33.33 x - 56 x + 31


# Inverse Interpolation

In [117]:
n = int(input('Enter number of data points: '))
x = np.zeros((n))
y = np.zeros((n))
print('Enter data for x and y: ')
for i in range(n):
    x[i] = float(input( 'x['+str(i)+']='))
print(x)
for i in range(n):
    y[i] = float(input( 'y['+str(i)+']='))
print(y)

yp = float(input('Enter interpolation point: '))
xp = 0

for i in range(n): 
    p = 1 
    for j in range(n):
        if i != j:
            p = p * (yp - y[j])/(y[i] - y[j])    
    xp = xp + p * x[i]    

print('Interpolated value at %.3f is %.3f.' % (yp,xp))

Enter data for x and y: 
[ 5.  6.  9. 11.]
[12. 13. 14. 16.]
Interpolated value at 15.000 is 11.500.
